# An all-sky ALMA Archive search for some target lines within a given redshift range
## (Authors: Toby Brown and Nathan Brunetti)

## Uses the `ALMAxmatch` module's `runQueriesWithLines` method

### Import the `ALMAxmatch` module
- This requires installing the latest code branch for the `astroquery` moduledue to a bug in the release date format that should be fixed in the next release.

## Install  "bleeding edge" version of `astroquery`
In the short-term it is necessary to install the "bleeding edge" version of `astroquery` to avoid a bug in the release date column. To do so, run the following commands in a terminal in a directory you do not mind downloading ~100 MB of code
```
git clone https://github.com/astropy/astroquery.git
cd astroquery
python setup.py build
python setup.py install
```

### Import the `ALMAxmatch` tool

In [5]:
import os
import sys
sys.path.insert(0, os.path.abspath('..'))
from ALMAxmatch import archiveSearch

## 1. Define lines of interest and redshift range

### Lines:

1. $^{12}$CO ($J=1-0$) @ 115.27120180 GHz rest frequency
2. $^{12}$CO ($J=2-1$) @ 230.538 GHz rest frequency

In [ ]:
# rest frame frequencies [These have to be in a bug]
rf_12CO_10 = [115.27120180] # 12CO J=1-0
rf_12CO_21 = [230.538] # 12CO J=2-1

line_12CO_10 = ["12CO(1-0)"]
line_12CO_21 = ["12CO(2-1)"]

### Search Redshift Range:

Search for the following lines at $0.01 \leq z \leq 0.06$:

In [7]:
# search redshift range
redshift_range = [0.01,0.06]

## 2. Run the archive search


### What does `runQueriesWithLines` do?
1. Calculate min and max frequencies for the target lines in the redshift range
1. Query the service using `astroquery.alma.Alma.query` with the min and max frequencies for the target lines and return a table object, selecting only science data
1. Cross-match these observations with NED, returning the results in the following astropy tables:

    `archiveSearch.queryResults`: ALMA Archive information for observations that match a NED object name and have a redshift, with flags for each line specifying if the spectral windows cover the line frequency at the object's redshift.

    `archiveSearch.queryResultsNoNED`: Observations that did not have a match in NED, based on name.

    `archiveSearch.queryResultsNoNEDz`: Observations that match a NED object name but do not have a redshift.

In [9]:
# Initialise the search objects
allSkyLinesQuery_10 = archiveSearch(allSky=True)
allSkyLinesQuery_21 = archiveSearch(allSky=True)

**Warning** Running this query takes about 12 minutes to run, so make sure you have something else to do while you wait (ALMA archive pro-tip: use this time to give your mom a quick call, you know it's been too long).

In [10]:
# CO 1-0
allSkyLinesQuery_10.runQueriesWithLines(restFreqs=rf_12CO_10,
                                     redshiftRange=redshift_range,
                                     lineNames=line_12CO_10, science=True)

NED cross matching: 100%|██████████| 3778/3778 [01:24<00:00, 44.92 source/s]


In [12]:
# CO 2-1
allSkyLinesQuery_21.runQueriesWithLines(restFreqs=rf_12CO_21,
                                     redshiftRange=redshift_range,
                                     lineNames=line_12CO_10, science=True)

NED cross matching:  18%|█▊        | 2963/16892 [08:16<12:48:20,  3.31s/ source]

KeyboardInterrupt: 

In [ ]:
allSkyLinesQuery.queryResults['All sky'].show_in_notebook()

In [ ]:
print("There are", len(allSkyLinesQuery.queryResults['All sky']), "observations of 12CO(1-0) OR (2-1) in the target redshift range")

Because our search results have accompanying archival and NED data, we can take a better look

## 3. Take a look at your results

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10,5))
plt.hist(allSkyLinesQuery.queryResults['All sky']['NED Redshift'])
plt.xlabel("redshift")
plt.show()

In [ ]:
_CO_observed = allSkyLinesQuery.queryResults['All sky'][lineNames[0]]
_13CO_observed = allSkyLinesQuery.queryResults['All sky'][lineNames[1]]

perfomance = [len(C18O_observed[C18O_observed==True]),
               len(_13CO_observed[_13CO_observed==True]),
               len(_12CO_observed[_12CO_observed==True])]

xpos = [0,1,2]

plt.figure(figsize=(10,5))
plt.bar(xpos, perfomance)
plt.xticks(xpos, lineNames, fontsize=15)
plt.ylabel('No. Observations')

### Group by NED source name

In [ ]:
observations = allSkyLinesQuery.queryResults['All sky'].group_by('NED source name')

Iterate over the group sub-tables and corresponding keys with:

In [ ]:
n = 0
for key, group in zip(observations.groups.keys, observations.groups):
    # if all three lines detected
    if ((True in group['12CO J=1-0']) & 
        (True in group['13CO J=1-0']) &
        (True in group['C18O J=1-0'])):
        
        print('****** {0} *******'.format(key['NED source name']))
        display(group)
        print('')
        n= n+1


In [ ]:
print("There are", n, "galaxies in the target redshift range with 12CO, 13CO, AND C18O observations.")